Import

In [11]:
import pandas as pd
import class_def as cd
import re
import random
import openpyxl
import numpy as np
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles.borders import Border, Side

def printErrorMsg(fileName):
    print(fileName)
    print('Press any key to continue ...')
    input()
    exit()

print('Reading data ...')

try:
    df= pd.read_excel('Input/Specific Examer.xlsx')
except:
    printErrorMsg('Can\'t find file \'Specific Examer.xlsx\'!')

MAIN_EXAMER_OF_ENG_SPEAKING = [x for x in df['English Speaking\n主考官'].tolist() if x == x]
ORAL_EXAMER_OF_ENG_SPEAKING = [x for x in df['English Speaking\nOral 考官'].tolist() if x == x]
MAIN_EXAMER_OF_ENG_LISTENING = [x for x in df['English Listening\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_CHIN_SPEAKING = [x for x in df['中文説話\n主考官'].tolist() if x == x]
ORAL_EXAMER_OF_CHIN_SPEAKING = [x for x in df['中文説話\nOral 考官'].tolist() if x == x]
MAIN_EXAMER_OF_CHIN_LISTENING = [x for x in df['中文聆聽\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_PTH = [x for x in df['普通話\n主考官'].tolist() if x == x]
MAIN_EXAMER_OF_VA = [x for x in df['VA\n主考官'].tolist() if x == x]
FOREIGN_TEACHER = [x for x in df['外籍老師'].tolist() if x == x]
SPECIAL_TIME_TEACHER = [x for x in df['特殊時數'].tolist() if x == x]
SPECIAL_TA = [x for x in df['特殊TA'].tolist() if x == x]
ENG_SPEAKING_HALL_TA = [x for x in df['English Speaking HALL TA'].tolist() if x == x]
SPEAKING_PR_TA = [x for x in df['English Speaking preparation room TA'].tolist() if x == x]

tmp = {}
for examer in MAIN_EXAMER_OF_VA:
    examer = examer.replace(' ', '')
    tmp[int(examer[:examer.index(':')])] = examer[examer.index(':')+1:]
MAIN_EXAMER_OF_VA = tmp

tmp = {}
for examer in SPECIAL_TIME_TEACHER:
    examer = examer.replace(' ', '')
    tmp[examer[:examer.index(':')]] = float(examer[examer.index(':')+1:])
SPECIAL_TIME_TEACHER = tmp

CANT_BE_EXAMER = [x for x in df['不能監考\n(校長)'].tolist() if x == x]

TA_DATA = []
for ta in [x for x in df['TA'].tolist() if x == x]:
    TA_DATA.append(cd.TA(name=ta))

try:
    df= pd.read_excel('Input/Other Info.xlsx', sheet_name='科目名稱對照')
except:
    printErrorMsg('Can\'t find file \'Other Info.xlsx\'!')

SUBJECT_NAME_DICT = {[x for x in df['科目中文名'].tolist() if x == x][i] : [x for x in df['科目縮寫'].tolist() if x == x][i].replace(' ','').split(',') for i in range(len([x for x in df['科目中文名'].tolist() if x == x]))}

df= pd.read_excel('Input/Other Info.xlsx', sheet_name='班別科室對照')
CLASS_DICT = {str([x for x in df['課室'].tolist() if x == x][i]) : [x for x in df['班別'].tolist() if x == x][i] for i in range(len([x for x in df['課室'].tolist() if x == x]))}

df= pd.read_excel('Input/Other Info.xlsx', sheet_name='班主任')
CLASS_TEACHER = {[x for x in df['班主任'].tolist() if x == x][i] : [x for x in df['班別'].tolist() if x == x][i] for i in range(len([x for x in df['班主任'].tolist() if x == x]))}

Reading data ...


Reading exam timetable data

In [12]:
try:
    df= pd.read_excel('Input/Exam Timetable.xlsx', skiprows=[0], usecols=lambda x: 'Unnamed' not in x)
except:
    printErrorMsg('Can\'t find file \'Exam Timetable.xlsx\'!')

ET_DATA = []
for date in df.columns:
    if date[-2:-1] in ['一', '二', '三', '四', '五', '六', '日']:
        ET_DATA.append(cd.exam(examDate=date))
    else:
        print('Exam Timetable: Date Formate Error!')
    
for exam in ET_DATA:
    exam.subjects = []
    exam.noExam = []
    listedColum = df[exam.examDate].tolist()
    listedColum.insert(0, exam.examDate)
    form = 0
    for i in range(len(listedColum)-1):
        if listedColum[i] == exam.examDate:
            form += 1
            if listedColum[i+1] == '上課':
                exam.noExam.append(form)
    listedColum = [x for x in listedColum if x == x] #remove nan
    listedColum = list(filter(lambda i: i != '上課', listedColum)) #remove '上課'
    i = 0
    form = 0
    while i < len(listedColum):
        if listedColum[i] == exam.examDate:
            form += 1
            i += 1
        else:
            exam.subjects.append(cd.subject(name = listedColum[i], timeLimit = listedColum[i+1], room = listedColum[i+2], period = listedColum[i+3], form=form, parent=exam))
            i += 4



Reading teacher time table data

In [13]:
def appendSubjectsAndClasses(teacher, lessonName):
    s = re.compile(r'[1-6][A-F] [A-Za-z0-9]+').findall(lessonName)
    c = re.compile(r'[1-6][A-F]').findall(lessonName)
    if len(s) > 0:
        if s[0][3:] not in [key for key in teacher.teachedSubjectsAndClasses]:
            teacher.teachedSubjectsAndClasses[s[0][3:]] = []
        if not set(c).issubset(teacher.teachedSubjectsAndClasses[s[0][3:]]):
            teacher.teachedSubjectsAndClasses[s[0][3:]] += c
        return [c, s[0][3:]]
    else:
        return None

try:
    sheets = pd.ExcelFile('Input/Teacher Timetable.xlsx')
    timeSlot = pd.read_excel('Input/Teacher Timetable.xlsx', skiprows=[0,1])['Unnamed: 0'].tolist()
except:
    printErrorMsg('Can\'t find file \'Teacher Timetable.xlsx\'!')

TT_DATA = []
dateDict = {'Mon' : '一', 'Tue' : '二', 'Wed' : '三', 'Thu' : '四', 'Fri' : '五'}
for sheetName in sheets.sheet_names:
    if sheetName not in CANT_BE_EXAMER:
        TT_DATA.append(cd.teacher(sheetName))
for teacher in TT_DATA:
    teacher.lessons = {}
    teacher.totalTime = 0
    teacher.teachedSubjectsAndClasses = {}
    teacher.exams = {}
    df = pd.read_excel('Input/Teacher Timetable.xlsx', skiprows=[0,1], sheet_name=teacher.name, usecols=lambda x: 'Unnamed' not in x)
    for date in df.columns:
        listedColum = df[date].tolist()
        teacher.lessons[dateDict[date]] = {}
        for i in range(len(listedColum)):
            if 'unch' in str(listedColum[i]):
                break
            elif 'orning' in str(listedColum[i]) and teacher.name in [key for key in CLASS_TEACHER]:
                teacher.lessons[dateDict[date]][timeSlot[i]] = [[CLASS_TEACHER[teacher.name]], '班主任']
            elif re.match('[1-6][A-F]', str(listedColum[i])) != None:
                teacher.lessons[dateDict[date]][timeSlot[i]] = appendSubjectsAndClasses(teacher, listedColum[i])

for teacher in TT_DATA:
    teacher.totalTime = 0
    for exam in ET_DATA:
        for needLessonForms in exam.noExam:
            for key in teacher.lessons[exam.examDate[-2:-1]]:
                if teacher.lessons[exam.examDate[-2:-1]][key][0][0][0] == str(needLessonForms) and teacher.lessons[exam.examDate[-2:-1]][key][1] != '班主任':
                    teacher.lessonTime += 35
    teacher.totalTime = teacher.lessonTime

In [14]:
for teacher in TT_DATA:
    print(teacher.name + ': ' + str(teacher.lessons))

AO: {'一': {'8:35 AM\n-9:10 AM': [['1C', '1D', '2B'], 'ENG'], '9:10 AM\n-9:45 AM': [['1C', '1D', '2B'], 'ENG'], '10:05 AM\n-10:40 AM': [['2C', '2D'], 'ENG'], '10:40 AM\n-11:15 AM': [['4A', '4B', '4C', '4D', '4E'], 'ENG'], '11:15 AM\n-11:50 AM': [['4A', '4B', '4C', '4D', '4E'], 'ENG']}, '二': {'10:40 AM\n-11:15 AM': [['1A', '1B'], 'ENG'], '12:10 PM\n-12:45 PM': [['2C', '2D'], 'ENG'], '12:45 PM\n-1:20 PM': [['2C', '2D'], 'ENG']}, '三': {'10:05 AM\n-10:40 AM': [['1C'], 'ENG'], '11:15 AM\n-11:50 AM': [['1D'], 'ENG'], '12:10 PM\n-12:45 PM': [['2C', '2D'], 'ENG'], '12:45 PM\n-1:20 PM': [['2C', '2D'], 'ENG']}, '四': {'8:35 AM\n-9:10 AM': [['2C', '2D'], 'ENG'], '9:10 AM\n-9:45 AM': [['2C', '2D'], 'ENG'], '11:15 AM\n-11:50 AM': [['1A', '1B'], 'ENG'], '12:10 PM\n-12:45 PM': [['1C', '1D', '2B'], 'ENG'], '12:45 PM\n-1:20 PM': [['1C', '1D', '2B'], 'ENG']}, '五': {'8:35 AM\n-9:10 AM': [['2C', '2D'], 'ENG'], '9:10 AM\n-9:45 AM': [['2C', '2D'], 'ENG'], '11:15 AM\n-11:50 AM': [['1A', '1B'], 'ENG'], '12:10 P

In [15]:
for teacher in TT_DATA:
    print(teacher.name + ': ' + str(teacher.teachedSubjectsAndClasses))

AO: {'ENG': ['1C', '1D', '2B', '2C', '2D', '4A', '4B', '4C', '4D', '4E', '1A', '1B']}
ASH: {'PE': ['3A', '3D', '4D', '3B', '3E', '6C', '6E', '2B', '2D', '4B', '4E', '1B', '1D', '1A', '1C', '5C', '5E', '6A', '6B', '6D']}
CHP: {'PTH': ['1B', '1C'], 'CHIN': ['1C', '5A', '3E']}
CHM: {'MUS': ['1B', '2B', '1D'], 'CHIN': ['6B']}
CMW: {'ENG': ['6A', '6B', '4A', '4B', '4C', '4D', '4E', '6C', '6D', '5A', '5B', '5C', '5D', '5E']}
CTC: {'CS': ['4D', '5E', '4A', '5B'], 'LIBS': ['6D', '6C']}
CHCH: {'ENG': ['1C', '1D', '2B', '1A', '1B', '6C', '6D']}
CWYU: {'CHIN': ['3A', '5D', '6A', '5A', '5B', '5C', '5D', '5E']}
CLC: {'ECON': ['5A', '5B', '5C', '5D', '5E', '6C', '4C']}
CLY: {'MACO': ['4C', '6A'], 'MATH': ['1B']}
CFL: {'GEOG': ['3B', '3E', '4A', '3A', '3D'], 'CHIS': ['1A', '1D', '1C', '2B', '2A', '1B', '2D']}
FWC: {'CL': ['3E', '3D', '3C', '3B', '2C'], 'ICT': ['5D', '5E'], 'MEd': ['2C']}
FCH: {'BIO': ['4A', '4B', '4C', '4E', '4D', '6D', '3C', '3D']}
HWL: {'MAM2': ['4A', '4B', '4C', '4D', '4E', '2D'],

Calculate avg. time

In [16]:
AVG_TIME = 0
for exam in ET_DATA:
    for subject in exam.subjects:
        if subject.room[0] == 'HALL':
            AVG_TIME += subject.timeLimit[0] * (len(subject.room) - 1)
        else:
            AVG_TIME += subject.timeLimit[0] * len(subject.room)
            

for teacher in TT_DATA:
    if teacher.name not in FOREIGN_TEACHER:
        AVG_TIME += teacher.totalTime
AVG_TIME /= (len(TT_DATA) - len(FOREIGN_TEACHER))

Distributing teachers to exams

In [17]:
def checkTime(examTime, lessonTime):
    time1 = []
    time2 = []
    numPattern = re.compile(r'\d+')
    time1.append(int(numPattern.findall(examTime)[0])*60+int(numPattern.findall(examTime)[1]))
    time1.append((int(numPattern.findall(examTime)[-2]) + (12 if (re.search( r'p', examTime, re.I) and len(numPattern.findall(examTime)[-2]) < 2) else 0))*60+int(numPattern.findall(examTime)[-1]))
    time2.append(int(numPattern.findall(lessonTime)[0])*60+int(numPattern.findall(lessonTime)[1]))
    time2.append((int(numPattern.findall(lessonTime)[-2]) + (12 if (re.search( r'p', lessonTime, re.I) and len(numPattern.findall(lessonTime)[-2]) < 2) else 0))*60+int(numPattern.findall(lessonTime)[-1]))
    if (time1[0] > time2[1]) or (time1[1] < time2[0]):
        return True
    else:
        return False


def findAvalibleTeachers(subject, specificExamer=None, needCheck = False):
    avalibleTeachersList = []
    teacherData = []
    if specificExamer != None:
        for teacherNames in specificExamer:
            teacherData.append(findParentObj(TT_DATA, teacherNames))
    else:
        teacherData = TT_DATA
    for teacher in teacherData:
        avalible = True
        if teacher.name in FOREIGN_TEACHER and specificExamer != FOREIGN_TEACHER:
            avalible = False
        if specificExamer == None and teacher.name in SPECIAL_TIME_TEACHER and (teacher.totalTime >= SPECIAL_TIME_TEACHER[teacher.name] * AVG_TIME):
            avalible = False
        if len(subject.parent.noExam) > 0:
            for value in teacher.lessons[subject.parent.examDate[-2:-1]].values(): 
                value = value[0][0][0]
                if value in subject.parent.noExam:
                    for lessonTime in [key for key in teacher.lessons[subject.parent.examDate[-2:-1]] if teacher.lessons[subject.parent.examDate[-2:-1]][key][0][0][0] == value]:
                        if avalible:
                            avalible = checkTime(subject.period, lessonTime)
                        else:
                            break
                if not avalible:
                    break
        if subject.parent.examDate in [key for key in teacher.exams]:
            for examTime in map(lambda x: x.period, teacher.exams[subject.parent.examDate]):
                if avalible:
                    avalible = checkTime(subject.period, examTime)
                else:
                    break
        
        if needCheck:
            tmp = subject.name[:subject.name.index(' ') if ' ' in subject.name else len(subject.name)]
            if tmp in [key for key in SUBJECT_NAME_DICT]:
                for subjectName in SUBJECT_NAME_DICT[tmp]:
                    if subjectName in [key for key in teacher.teachedSubjectsAndClasses]:
                        if len(subject.room) > 2:
                            if CLASS_DICT[subject.room[subject.teachers.index('')]] in teacher.teachedSubjectsAndClasses[subjectName]:
                                avalible = False
                        else:
                            if str(subject.form) in list(map(lambda x: x[0], teacher.teachedSubjectsAndClasses[subjectName])):
                                avalible = False

        if avalible:
            avalibleTeachersList.append(teacher)
            
    avalibleTeachersList.sort(key=lambda x: x.totalTime, reverse=False)
    return avalibleTeachersList[0]
    
    

def findParentObj(data, name):
    return data[list(map(lambda x : x.name == name, data)).index(True)]

def appendTeachers(i, subject, avalibleTeacher, isOral=False):
    if subject.teachers[i] != '':
        return
    subject.teachers[i] = avalibleTeacher.name
    if not isOral:
        avalibleTeacher.totalTime += subject.timeLimit[0]
    else:
        avalibleTeacher.totalTime += subject.timeLimit[1]
    if subject.parent.examDate not in [key for key in avalibleTeacher.exams]:
        avalibleTeacher.exams[subject.parent.examDate] = []
    avalibleTeacher.exams[subject.parent.examDate].append(cd.examDetails(subject.name, subject.period, subject.room[i]))

def appendTA(i, subject, specific=None):
    avalibleTAList = []
    tmp = []
    if specific != None:
        for TAName in specific:
            tmp.append(findParentObj(TA_DATA, TAName))
    else:
        tmp = TA_DATA

    for TA in tmp:
        avalible = True
        if subject.parent.examDate in [key for key in TA.exams]:
            for examTime in map(lambda x: x.period, TA.exams[subject.parent.examDate]):
                if avalible:
                    avalible = checkTime(subject.period, examTime)
                else:
                    break
        
        if avalible:
            avalibleTAList.append(TA)

    avalibleTAList.sort(key=lambda x: x.totalTime, reverse=False)
    avalibleTA = avalibleTAList[0]
    subject.teachers[i] = avalibleTA.name
    avalibleTA.totalTime += subject.timeLimit[0]
    if subject.parent.examDate not in [key for key in avalibleTA.exams]:
        avalibleTA.exams[subject.parent.examDate] = []
    avalibleTA.exams[subject.parent.examDate].append(cd.examDetails(subject.name, subject.period, subject.room[i]))

def appendRemaining(subject):
    appendTA(len(subject.teachers)-1, subject)
    if subject.teachers[len(subject.teachers)-1] in SPECIAL_TA:
        appendTA(len(subject.teachers)-2, subject)
    for i in range(len(list(filter(lambda x: x == '', subject.teachers)))):
        appendTeachers(subject.teachers.index(''), subject, findAvalibleTeachers(subject))
print('Processing ...')

for exam in ET_DATA:
    for subject in exam.subjects:
        if 'peaking' in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, MAIN_EXAMER_OF_ENG_SPEAKING))
            for i in range(1,len(list(filter(lambda x: x == 'HALL', subject.room)))):
                appendTA(i, subject, ENG_SPEAKING_HALL_TA)
            for i in range(subject.teachers.index(''),len(subject.room)):
                if subject.room[i][-2] == 'p':
                    appendTA(i, subject, SPEAKING_PR_TA)
            for i in range(len(FOREIGN_TEACHER)):
                appendTeachers(subject.teachers.index(''), subject, findAvalibleTeachers(subject, FOREIGN_TEACHER), isOral=True)
            for i in range(len(list(filter(lambda x: x == '', subject.teachers)))):
                appendTeachers(subject.teachers.index(''), subject, findAvalibleTeachers(subject, ORAL_EXAMER_OF_ENG_SPEAKING), isOral=True)
        elif 'istening' in subject.name and 'TSA' not in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, MAIN_EXAMER_OF_ENG_LISTENING))
            appendTeachers(1, subject, findAvalibleTeachers(subject))
            for i in range(2,len(subject.room)):
                appendTA(i, subject)
        elif '說話' in subject.name or '説話' in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, MAIN_EXAMER_OF_CHIN_SPEAKING))
            for i in range(1,len(list(filter(lambda x: x == 'HALL', subject.room)))):
                appendTA(i, subject)
            for i in range(subject.teachers.index(''),len(subject.room)):
                if subject.room[i][-2] == 'p':
                    appendTA(i, subject, SPEAKING_PR_TA)
            for i in range(len(list(filter(lambda x: x == '', subject.teachers)))):
                appendTeachers(subject.teachers.index(''), subject, findAvalibleTeachers(subject, ORAL_EXAMER_OF_CHIN_SPEAKING), isOral=True)
        elif '普通話' in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, MAIN_EXAMER_OF_PTH))
            appendRemaining(subject)
        elif '聆聽' in subject.name and 'TSA' not in subject.name and '普通話' not in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, MAIN_EXAMER_OF_CHIN_LISTENING))
            appendRemaining(subject)
        elif '視覺藝術' in subject.name:
            appendTeachers(0, subject, findAvalibleTeachers(subject, [MAIN_EXAMER_OF_VA[subject.form]]))

for subject in sorted(list(filter(lambda x: '' in x.teachers, list(np.concatenate(list(map(lambda x: x.subjects, ET_DATA))).flat))), key=lambda x: x.timeLimit[0], reverse=True):
    if 'HALL' in subject.room:
        appendTeachers(0, subject, findAvalibleTeachers(subject))   
        appendRemaining(subject)
    else:
        for i in range(0,len(subject.room)):
            appendTeachers(i, subject, findAvalibleTeachers(subject, needCheck=True))
    

Processing ...


Print out results

In [18]:
print('TA總監考時間')
for ta in TA_DATA:
    print(ta.name + ': ' + str(ta.totalTime) + ' mins')

TA總監考時間
LKHE: 440 mins
LLS: 440 mins
LYT: 450 mins
LCHU: 455 mins
MLM: 460 mins
TSYE: 470 mins


In [19]:
for teacher in TT_DATA:
    print(teacher.name + ': ' + str(teacher.exams))

for TA in TA_DATA:
    print(TA.name + ': ' + str(TA.exams))

AO: {'15/06/2023 (四)': [<class_def.examDetails object at 0x000001A663C89790>], '16/06/2023 (五)': [<class_def.examDetails object at 0x000001A663C8AD90>], '19/06/2023 (一)': [<class_def.examDetails object at 0x000001A663C8ACD0>], '20/06/2023 (二)': [<class_def.examDetails object at 0x000001A6643A7910>], '23/06/2023 (五)': [<class_def.examDetails object at 0x000001A662B6F410>]}
ASH: {'21/06/2023 (三)': [<class_def.examDetails object at 0x000001A662B6DF10>], '19/06/2023 (一)': [<class_def.examDetails object at 0x000001A662B6CF10>], '16/06/2023 (五)': [<class_def.examDetails object at 0x000001A663E89B50>], '14/06/2023 (三)': [<class_def.examDetails object at 0x000001A664234E10>], '15/06/2023 (四)': [<class_def.examDetails object at 0x000001A663CE2910>], '23/06/2023 (五)': [<class_def.examDetails object at 0x000001A663CE3D10>]}
CHP: {'21/06/2023 (三)': [<class_def.examDetails object at 0x000001A663F19B90>, <class_def.examDetails object at 0x000001A663E8BC90>], '20/06/2023 (二)': [<class_def.examDetails

Output to xlxs

In [20]:
workbook = openpyxl.Workbook()
sheet = workbook.worksheets[0]
sheet.title = '考試時間表 + 監考'
formDict = { 1 : '中一級', 2 : '中二級', 3 : '中三級', 4 : '中四級', 5 : '中五級', 6 : '中六級'}

greyFill = PatternFill(patternType='solid', fgColor=Color(rgb='D9D9D9'))
yellowFill = PatternFill(patternType='solid', fgColor=Color(rgb='FFFF00'))
orangeFill = PatternFill(patternType='solid', fgColor=Color(rgb='FFC000'))
mediumBorder = Border(left=Side(style='medium'), 
                     right=Side(style='medium'), 
                     top=Side(style='medium'), 
                     bottom=Side(style='medium'))
thinBorder = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))


for i in range(ET_DATA[0].subjects[-1].form):
    sheet.cell(row = sheet.max_row+2, column = 1).value = formDict[i+1]
    sheet.cell(row = sheet.max_row+1, column = 1).border = mediumBorder
    top = sheet.max_row

    subjectListFilteredByForm = list(map(lambda x: list(filter(lambda y: y.form == i+1, x)) ,map(lambda x: x.subjects, ET_DATA)))
    maxRowLength = [''] * len(sorted(subjectListFilteredByForm, key=lambda z: len(z), reverse=True)[0])
    
    for j in range(len(maxRowLength)):
        maxRowLength[j] = max(list(map(lambda x: len(x[j].room) if j < len(x) else 0, subjectListFilteredByForm)))
    for j in range(len(maxRowLength)):
        if j == 0:
            tmp = sheet.max_row
        else:
            tmp = sheet.max_row+1
        for col in range(len(ET_DATA)+1):
            for row in range(maxRowLength[j]+3):
                sheet.cell(row = tmp+row+1, column = col+1).border = mediumBorder
                sheet.cell(row = tmp+row+1, column = col+1).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')
                
    for col, exam in enumerate(ET_DATA,start=2):
        sheet.cell(row = top, column = col).value = exam.examDate
        sheet.cell(row = top, column = col).border = mediumBorder
        sheet.cell(row = top, column = col).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')
        sheet.cell(row = top, column = col).font = Font(bold=True)
        sheet.column_dimensions[get_column_letter(col)].width = 17
        current_row = top+1
        for k, subject in enumerate(list(filter(lambda x: x.form == i+1, exam.subjects))):
            sheet.cell(row = current_row, column = col).value = subject.name
            sheet.cell(row = current_row, column = col).font = Font(bold=True)
            sheet.cell(row = current_row, column = 1).value = '科 目'
            sheet.cell(row = current_row, column = 1).fill = orangeFill
            sheet.cell(row = current_row, column = col).fill = orangeFill

            sheet.cell(row = current_row+1, column = col).value = '/'.join(list(map(lambda x: str(x) ,subject.timeLimit)))
            sheet.cell(row = current_row+1, column = 1).value = '時 限'

            sheet.cell(row = current_row+2, column = col).value = subject.period
            sheet.cell(row = current_row+2, column = 1).value = '應考時間'

            current_row += 3
            if sheet.cell(row = current_row, column = 1).value == None:
                sheet.cell(row = current_row, column = 1).value = '應考試場\n監考'
                sheet.cell(row = current_row, column = 1).fill = yellowFill
                sheet.cell(row = current_row, column = 1).border = mediumBorder
                sheet.cell(row = current_row, column = 1).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')
            for j in range(maxRowLength[k]+1):
                if j < len(subject.room):
                    sheet.cell(row = current_row, column = col).value = subject.room[j] + ': ' + subject.teachers[j]
                    sheet.cell(row = current_row, column = col).fill = yellowFill
                    sheet.cell(row = current_row, column = col).alignment = Alignment(horizontal='center', wrapText=True, vertical = 'center')
                    sheet.cell(row = current_row, column = col).border = mediumBorder
                else:
                    sheet.cell(row = current_row, column = col).value = None
                current_row += 1
            if len(subject.room) == maxRowLength[k]:
                sheet.merge_cells(start_row=(current_row-1-maxRowLength[k]), start_column=1, end_row=current_row-2, end_column=1)
    for y in range(2, sheet.max_column+1):
        for x in range(top, sheet.max_row+1):
            if sheet.cell(row = x, column = y).value == None and sheet.cell(row = x, column = y).border == mediumBorder:
                sheet.cell(row = x, column = y).fill = greyFill

workbook.create_sheet('老師上課 + 監考時數')
sheet2 = workbook.worksheets[1]
sheet2.cell(row = 1, column = 1).value = 'Teacher'
sheet2.cell(row = 1, column = 2).value = 'Ratio'
sheet2.cell(row = 1, column = 3).value = 'Lesson'
sheet2.cell(row = 1, column = 4).value = 'Exam'
sheet2.cell(row = 1, column = 5).value = 'Total'

for i, teacher in enumerate(TT_DATA, start=2):
    sheet2.cell(row = i, column = 1).value = teacher.name
    if teacher.name in [key for key in SPECIAL_TIME_TEACHER]:
        sheet2.cell(row = i, column = 2).value = SPECIAL_TIME_TEACHER[teacher.name]
        sheet2.cell(row = i, column = 2).fill = yellowFill
    else:
        sheet2.cell(row = i, column = 2).value = 1
    sheet2.cell(row = i, column = 3).value = teacher.lessonTime
    sheet2.cell(row = i, column = 4).value = teacher.totalTime - teacher.lessonTime
    sheet2.cell(row = i, column = 5).value = teacher.totalTime
    
for col in range(sheet2.max_column):
    for row in range(sheet2.max_row):
        sheet2.cell(row=row+1, column=col+1).border = thinBorder
        
ignoreTeacher = [key for key in SPECIAL_TIME_TEACHER] + FOREIGN_TEACHER

tmp = sheet2.max_row+2
sheet2.cell(row=tmp, column=1).value = 'Avg.'
sheet2.cell(row=tmp, column=2).value = round(np.average(list(map(lambda x: x.totalTime, filter(lambda x: x.name not in ignoreTeacher, TT_DATA)))))

workbook.save('監考時間表.xlsx')